In [ ]:
import gzip
import glob
import json
import pandas as pd
import tensorflow as tf

In [ ]:
!pip install -q tensorflow==2.4.1

In [ ]:
!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

In [ ]:
!pip install -Uqr models/official/requirements.txt

In [ ]:
covid_files_path = r'path'

covid_files = glob.glob(covid_files_path)
train_count = 0
test_count = 0

train_data = []
test_data = []
test_labels = []
train_labels = []

In [ ]:
for i in range(len(covid_files)):
    print("opening file", covid_files[i])
    while train_count < 25001:
        try:
            with gzip.open(covid_files[i],'r') as fin:
                for line in fin:
                    tweet = json.loads(line)
                    if train_count <25001:
                        if tweet['truncated'] == True:
                            train_data.append(tweet['extended_tweet']['full_text'])
                        else:
                            train_data.append(tweet['text'])
                        train_labels.append(1)
                        train_count += 1
        except:
            print("Faulty file ", files[i])
            
    while test_count < 10001:
        try:
            with gzip.open(covid_files[i],'r') as fin:
                for line in fin:
                    tweet = json.loads(line)
                    if test_count <10001: 
                        test_data.append()
                        test_count += 1
        except:
            print("Faulty file ", files[i])
        

In [ ]:
noncovid_files_path = r'path2'
noncovid_files = glob.glob(noncovid_files_path)

In [ ]:
for i in range(len(noncovid_files)):
    print("opening file", noncovid_files[i])
    while train_count < 25001:
        try:
            with gzip.open(noncovid_files[i],'r') as fin:
                for line in fin:
                    tweet = json.loads(line)
                    if train_count <25001: 
                        train_data.append(tweet['text'])
                        train_labels.append(0)
                        train_count += 1
        except:
            print("Faulty file ", files[i])
            
    while test_count < 10001:
        try:
            with gzip.open(noncovid_files[i],'r') as fin:
                for line in fin:
                    tweet = json.loads(line)
                    if test_count <10001: 
                        test_data.append()
                        test_count += 1
        except:
            print("Faulty file ", files[i])

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_data_df = pd.Dataframe(train_data, columns = ['Train Data'])
test_data_df = pd.Dataframe(test_data, columns = ['Test Data'])
train_labels_df = pd.Dataframe(train_data, columns = ['Train Labels'])
test_labels_df = pd.Dataframe(train_data, columns = ['Test Labels'])

train_data_labels = pd.concat([train_data_df, train_labels_df], axis=1)
test_data_labels = pd.concat([test_data_df, test_labels_df], axis=1)

In [ ]:
train_df, remaining = train_test_split(train_data_labels, random_state = 42, train_size = 0.0075, stratify = df.target.values)
valid_df, _ = train_test_split(remaining, random_state = 42, train_size = 0.00075, stratify = remaining.target.values)
train_df.shape, valid_df.shape

In [ ]:
with tf.device('/cpu:0'):
    train_data = tf.data.Dataset.from_tensor_slices((train_df['question_text'].values, train_df['target'].values))
    valid_data = tf.data.Dataset.from_tensor_slices((valid_df.question_text.values, valid_df.target.values))
    
    for text, label  in train_data.take(1):
        print(text)
        print(label)

In [ ]:
label_list = [0, 1] # Label categories
max_seq_length = 128 # maximum length of (token) input sequences
train_batch_size = 32


# Get BERT layer and tokenizer:
# More details here: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2', trainable = True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize("hi, how are you doing?")

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.wordpiece_tokenizer.tokenize("hi, how are you doing?"))

In [ ]:
def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
    example = classifier_data_lib.InputExample(guid = None, text_a = text.numpy(), text_b = None, label = label.numpy())
    feature  = classifier_data_lib.convert_single_example(0, example, label_list, max_seq_length, tokenizer)
    return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

In [ ]:
def to_feature_map(text, label):
    input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp = [text, label], 
                                                                Tout = [tf.int32, tf.int32, tf.int32, tf.int32])
    input_ids.set_shape([max_seq_length])
    input_mask.set_shape([max_seq_length])
    segment_ids.set_shape([max_seq_length])
    label_id.set_shape([])
    x = {
      'input_word_ids': input_ids,
       'input_mask': input_mask,
       'input_type_ids': segment_ids
      }
    
    return (x, label_id)

In [ ]:
with tf.device('/cpu:0'):
    # train
    train_data = (train_data.map(to_feature_map,
                               num_parallel_calls = tf.data.experimental.AUTOTUNE)
                  .shuffle(1000)
                  .batch(32, drop_remainder = True)
                  .prefetch(tf.data.experimental.AUTOTUNE))
  

  # valid
    valid_data = (valid_data.map(to_feature_map,
                               num_parallel_calls = tf.data.experimental.AUTOTUNE)
                  .batch(32, drop_remainder = True)
                  .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
# Building the model
def create_model():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
    input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="input_type_ids")
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
    drop = tf.keras.layers.Dropout(0.4)(pooled_output)
    output = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output')(drop)
    model = tf.keras.Model(
        inputs = {
            'input_word_ids': input_word_ids,
            'input_mask': input_mask,
            'input_type_ids': input_type_ids
        },
        outputs  = output)
    return model

In [ ]:
model = create_model()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 2e-5),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy()])
model.summary()

In [ ]:
epochs = 4
history = model.fit(train_data,
                    validation_data = valid_data, 
                    epochs = epochs,
                    verbose = 1)